# Gemini 코드 마무리 복습 & RAG 진입 전 핵심 용어 정리


 ## 1. Gemini API
 ### 이미지생성 텍스트 변환

In [5]:
from google.colab import userdata


# Google API Key 불러오기
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [6]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

# client 인스턴스 생성
client = genai.Client(api_key=GOOGLE_API_KEY)

# 프롬프트를 입력해 모델이 생성한 이미지를 응답으로 받는다
response = client.models.generate_images(
    # imagen 모델 사용
    model="imagen-4.0-generate-preview-06-06",
    # 사용자 입력 프롬프트
    prompt="Cute quaka riding a red skateboard",
    config=types.GenerateImagesConfig(
        number_of_images=4, # 이미지 수량
    ),
)
for generated_image in response.generated_images:
    generated_image.image.show()

Output hidden; open in https://colab.research.google.com to view.

### 텍스트와 이미지를 함께 입력받아 새로운 이미지 생성하기

In [8]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

import PIL.Image

# 이미지 불러오기
image = PIL.Image.open("mark.jpg")

# client 인스턴스생성
client = genai.Client(api_key=GOOGLE_API_KEY)

text_input = ("Hi, This is a picture of me." "Can you add a tiger next to me?",)

# gemini-2.0-flash-preview-image-generation  하나의 모델로 여러가지를 생성할 수 있는 모델
response = client.models.generate_content(
    # 모델을 지정
    model="gemini-2.0-flash-preview-image-generation",
    # 텍스트 프롬프트와 기본 이미지를 리스트 형태로 함께 전달
    contents=[text_input, image],
    # 텍스트와 이미지를 모두 응답 받음
    config=types.GenerateContentConfig(response_modalities=["TEXT", "IMAGE"]),
)

# 결과 출력하기
for part in response.candidates[0].content.parts:
    # 텍스트 출력
    if part.text is not None:
        print(part.text)
    # 이미지 저장
    elif part.inline_data is not None:
        image = Image.open(BytesIO((part.inline_data.data)))
        image.save("result1.jpg")
        # image.show()

I will add a majestic Bengal tiger standing attentively next to you as you walk past rows of people wearing VR headsets.




In [10]:
# 텍스트와 이미지를 함께 입력받아 새로운 이미지 생성하기
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

client = genai.Client(api_key=GOOGLE_API_KEY)

# 기본 이미지 로드 (편집할 이미지)
base_image = Image.open("two_person.jpg")  # 실제 이미지 파일 경로로 변경


text_prompt = "이 이미지에 해변 배경을 추가하고, 하늘에 아름다운 노을을 그려주세요"


# 텍스트 프롬프트와 이미지를 함께 입력
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    # 텍스트 프롬프트와 기본 이미지를 리스트 형태로 함께 전달
    contents=[text_prompt, base_image],
    # 응답 형태를 설정
    config=types.GenerateContentConfig(
        #  AI가 설명 텍스트와 생성된 이미지를 모두 반환하도록 요청
        response_modalities=["TEXT", "IMAGE"]
    ),
)

# 응답 처리
for part in response.candidates[0].content.parts:
    # 생성된 텍스트 출력
    if part.text is not None:
        print("AI의 설명:")
        print(part.text)
    # 생성된 이미지 출력
    elif part.inline_data is not None:
        print("생성된 이미지:")
        generated_image = Image.open(BytesIO(part.inline_data.data))
        # 이미지 저장
        generated_image.save("result2.jpg")
        generated_image.show()  # 이미지 표시(팝업)

AI의 설명:
두 남녀 뒤로 넓은 모래사장과 잔잔한 푸른 바다가 펼쳐지고, 붉고 주황색으로 물든 아름다운 노을이 하늘을 가득 채우는 이미지를 생성해 드릴게요.


생성된 이미지:


### 텍스트 + 이미지 → 이미지 생성 (다양한 예제)

In [12]:
# 예제 1: 이미지 스타일 변경
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

client = genai.Client(api_key=GOOGLE_API_KEY)

# 원본 이미지
original_image = Image.open("mark.jpg")

# 스타일 변경 프롬프트
style_prompt = (
    "이 이미지를 만화 스타일로 변경해주세요. 밝고 화려한 색상을 사용해주세요."
)

# 메서드를 호출하여 AI에게 작업을 요청
response = client.models.generate_content(
    # 모델을 지정
    model="gemini-2.0-flash-preview-image-generation",
    # 프롬프트와 이미지를 모델에 전달
    contents=[style_prompt, original_image],
    # 응답으로 받을 데이터 형식 지정
    config=types.GenerateContentConfig(
        # 텍스트와 이미지 모두 응답
        response_modalities=["TEXT", "IMAGE"]
    ),
)

# 생성된 이미지와 설명 표시
for part in response.candidates[0].content.parts:
    # 텍스트 출력
    if part.text is not None:
        print("AI의 설명:", part.text)
    # 이미지 출력
    elif part.inline_data is not None:
        styled_image = Image.open(BytesIO(part.inline_data.data))
        styled_image.save("result3.jpg")

AI의 설명: 밝고 화려한 색상과 굵은 선을 사용하여 인물과 배경을 단순화한 만화 스타일로 이미지를 변환합니다. 마크 저커버그는 더욱 역동적인 포즈를 취하고, 청중들은 과장된 표정으로 VR 헤드셋을 착용하고 있는 모습으로 표현합니다.




### 이미지에 객체 추가,제거


In [14]:
# 예제 2: 이미지에 객체 추가/제거
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

client = genai.Client(api_key=GOOGLE_API_KEY)

base_image = Image.open("mark.jpg")

# 객체 추가 프롬프트
addition_prompt = """
이 이미지에 다음을 추가해주세요:
1. 배경에 아름다운 산과 나무들
2. 하늘에 흰 구름들
3. 인물 옆에 귀여운 강아지 한 마리
자연스럽게 합성해주세요.
"""

#  gemini-2.0-flash-preview-image-generation 모델 사용
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=[addition_prompt, base_image],
    config=types.GenerateContentConfig(response_modalities=["TEXT", "IMAGE"]),
)

# 결과 출력
for part in response.candidates[0].content.parts:
    if part.text is not None:
        print("편집 설명:", part.text)
    elif part.inline_data is not None:
        edited_image = Image.open(BytesIO(part.inline_data.data))
        edited_image.show()

### 코드실행

In [17]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

# AI에 작업을 요청
response = client.models.generate_content(
    # 모델 지정
    model="gemini-2.5-flash",
    # 프롬프트 전달
    contents="*을 피라미드 모양으로 출력해줘"
    "계산을 위한 코드를 생성하고 파이썬 코드의 내용을 출력해줘.",
    # 코드를 생성할 것을 요청
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

for part in response.candidates[0].content.parts:
    # text 부분 출력
    if part.text is not None:
        print(part.text)
        print("-" * 20)
    # code_executable 부분 출력. 마크다운에 파이썬 코드를 출력
    if part.executable_code is not None:
        print(part.executable_code.code)
        print("-" * 20)
    # code_execution_result 부분. 실행 결과를 출력
    if part.code_execution_result is not None:
        print(part.code_execution_result.output)

별 피라미드를 출력하는 파이썬 코드를 생성해 드리겠습니다. 피라미드의 높이를 몇 줄로 할지 알려주시면 정확한 코드를 생성해 드릴 수 있습니다.

우선, 5줄짜리 피라미드를 출력하는 코드를 예시로 보여드리겠습니다.

### 1. 피라미드 생성 Python 코드


--------------------
def print_pyramid(rows):
    """
    지정된 줄 수만큼 별(*) 피라미드를 출력합니다.

    Args:
        rows (int): 피라미드의 줄 수.
    """
    for i in range(1, rows + 1):
        # 각 줄의 시작 부분에 필요한 공백 수 계산
        # (총 너비 - 현재 줄의 별 개수) / 2
        # 총 너비는 마지막 줄의 별 개수와 동일: 2 * rows - 1
        num_spaces = rows - i
        
        # 현재 줄에 필요한 별 개수 계산
        # 1번째 줄: 1개, 2번째 줄: 3개, ... i번째 줄: 2*i - 1개
        num_stars = 2 * i - 1
        
        # 공백과 별을 조합하여 줄 생성
        line = " " * num_spaces + "*" * num_stars
        
        # 줄 출력
        print(line)

# 5줄짜리 피라미드 출력
print_pyramid(5)

--------------------
    *
   ***
  *****
 *******
*********

### 2. 코드 설명

위 파이썬 코드는 `print_pyramid(rows)` 함수를 사용하여 지정된 줄 수(`rows`)만큼 별 피라미드를 만듭니다.

*   **`for i in range(1, rows + 1):`**: 이 반복문은 피라미드의 각 줄을 순회합니다. `i`는 현재 줄 번호를 나타내며, 1부터 `rows`까지 증가합니다.
* 

## Google AI Studio 동영상 이해(요약)

In [33]:
import os

# Only for videos of size <20Mb
video_file_name = "sand-dam.mp4"
model_name = "models/gemini-2.5-flash"
prompt = "이 비디오의 내용을 3개 문장으로 요약해 주세요."
mime_type = "video/mp4"

In [34]:
# Vertex AI 클라이언트 초기화 (프로젝트 ID와 위치 설정)
project_id = os.getenv("GOOGLE_CLOUD_PROJECT_ID")  # .env 파일에 추가 필요
location = "us-central1"  # 또는 원하는 리전
model_name = "gemini-2.5-flash"  # Vertex AI 모델명 (models/ 접두사 제거)

In [35]:
from google.genai import types

# 바이너리로 읽어서 변수에 저장
video_bytes = open(video_file_name, "rb").read()

# 클라이언트 초기화
client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model=model_name,
    contents=types.Content(
        parts=[
            # 동영상을 바이너리로 입력
            types.Part(inline_data=types.Blob(data=video_bytes, mime_type=mime_type)),
            types.Part(text=prompt),
        ]
    ),
)

print(response.text)

강원도 산간 지역은 가뭄 시 식수 부족을 겪는데, 국내 연구진이 국내 최초로 모래 저장형 댐인 '샌드댐'을 개발했습니다. 이 댐은 물을 모래층에 저장하여 증발을 막고 거의 얼지 않아 가뭄에도 안정적으로 물을 공급하며, 자연 여과 기능으로 수질도 개선됩니다. 상수도 설치 비용의 10% 수준으로 건설할 수 있어, 전국 600여 곳의 물 소외 지역에 적용될 것으로 기대됩니다.


## OpenAI API


In [19]:
# Google API Key 불러오기
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [22]:
# OpenAI API를 사용하여 GPT-4o-mini 모델에게 요청
from openai import OpenAI

# Initialize the OpenAI client
# Make sure to set your OpenAI API key in the environment variable OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

# chating API 호
response = client.chat.completions.create(
    model="gpt-4o-mini",
    # 프롬프트 입력
    # messages는role과 content 키를 가진 딕셔너리로 구성됨
    messages=[
        {
            "role": "user",
            "content": "Write a one-sentence bedtime story about a unicorn.",
        }
    ],
)

# AI가 생성한 메시지 출력
print(response.choices[0].message.content)

Under the shimmering stars, a gentle unicorn named Luna danced through the enchanted forest, spreading dreams of joy and magic to every sleeping creature around her.


In [24]:
# OpenAI 호환성
# OpenAI 라이브러리로 작성된 코드를 크게 수정하지 않고도 Gemini 모델을 사용할 수 있도록 이러한 엔드포인트를 제공
from openai import OpenAI

# base_url을 다른 API 제공자의 엔드포인트로 변경
client = OpenAI(
    api_key=GOOGLE_API_KEY , base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# openai의 API 호출
response = client.chat.completions.create(
    # gemini 모델 지정
    model="gemini-2.5-flash",
    # 프롬프트 입력
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain to me how AI works"},
    ],
)

# 모델이 생성한 메시지 출력
print(response.choices[0].message)

ChatCompletionMessage(content='Artificial Intelligence (AI) isn\'t one single "thing" but rather a broad field of computer science focused on creating machines that can perform tasks that typically require human intelligence.\n\nAt its core, **AI works by enabling computers to learn from data, identify patterns, make decisions, and solve problems, often mimicking how a human brain might approach similar tasks.**\n\nLet\'s break down the "how" with an analogy:\n\n**Imagine teaching a child:**\n\n1.  **You show the child many pictures:** "This is a cat," "This is a dog," "This is a cat," "This is a dog." (This is like **Data Collection** for AI).\n2.  **The child starts to observe patterns:** Cats have pointy ears and whiskers; dogs are generally bigger and bark. (This is the **Algorithm/Model** in AI finding patterns).\n3.  **The child forms an internal "rule" or understanding:** "Okay, if it has pointy ears and meows, it\'s probably a cat." (This is the **Training Phase** where the AI 

## OpenAI 텍스트 생성

In [38]:
import base64
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

# 이미지 파일을 읽기
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# 이미지 경로
image_path = "movie.png"

# Base64 문자열로 변환
base64_image = encode_image(image_path)

response = client.chat.completions.create(
    # 모델 지정
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            # 입력할 텍스트와 이미지
            "content": [
                {"type": "text", "text": "이 포스터의 드라마 제목은?"},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
)

# AI가 생성한 메시지 출력
print(response.choices[0].message.content)

이 포스터의 드라마 제목은 "폭싹 속았수다"입니다.


In [40]:
from pathlib import Path
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

# Use current directory instead of __file__.parent since __file__ is not defined in notebooks
speech_file_path = Path("speech.mp3")

# 음성을 생성하는 API 호출
response = client.audio.speech.create(
    model="tts-1",
    # 음성 지정
    voice="nova",
    # input="Today is a wonderful day to build something people love!",
    input="오늘 날씨가 좋아요!",
)

# 오디오 파일 저장
with open(speech_file_path, "wb") as f:
    f.write(response.content)

## RAG 비법노트(교재 12p)
1. 문서 로드 : 외부문서(PDF, DOCS, CSV 등) 불러오기
2. 텍스트 분할 : 청크 단위로 나누기
3. 임베딩 : 청크를 벡터로 변환
4. 벡터 스토어 : 벡터를 데이터베이스에 저장 및 색인화
5. 리트리버 : 벡터 스토어에서 질문과 관련된 텍스트 검색하기
6. 프롬프트 : 검색된 텟스트와 질문을 조합해 LLM에 전달할 명령문 만들기
7. LLM : 프롬프트를 바탕으로 답변을 생성하거나 작업을 수행
8. 체인 : 모든 단계를 연결해 자동으로 흐름 구성하기

### 용어정리
- 청크(Chunk) : 큰 문서를 AI가 처리하기 쉬운 작은 단위로 나누는 작업의 결과물
- 체인 (Chain) : 여러 작업을 순차적으로 연결하여 복잡한 프로세스를 자동화하는 기능
- 리트리버 (Retriever) : 질문과 가장 관련성이 높은 문서를 벡터스토어에서 찾아내는 모듈
- RAG (Retrieval-Augmented Generation) : 외부 데이터 검색(Retrieval)과 생성(Generation)을 결합한 AI 시스템. LLM이 학습된 시점 이후의 최신 정보나 특정 분야의 도메인지식이 부족할 수 있다. 사용자가 답변을 생성하기 전에  리트리버를 통해 관련성 높은 외부문서를 검색하고 이를 바탕으로 답변을 생성하게 만든다.
